<a href="https://colab.research.google.com/github/Ardra-VV/Cosmic_Object_Identification/blob/main/CNN_IdentifyPlanet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16

In [2]:
plt.rc('figure', autolayout = True)
plt.rc('axes', labelweight = 'bold', titleweight = 'bold', titlesize = 18, titlepad = 10, labelsize = 'large')
plt.rc('image', cmap = 'magma')
warnings.filterwarnings("ignore")

In [3]:
def seed_set(seed = 31415):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC-OPS'] = '1'
seed_set(31415)

In [5]:
from google.colab import drive
drive.mount('/content/drive')
train_dir = '/content/drive/MyDrive/Identify_planet/train'
valid_dir = '/content/drive/MyDrive/Identify_planet/valid'
!pip install tensorflow_datasets
import tensorflow_datasets as tfds


print("Training directory:",train_dir)
print("Validation directory:",valid_dir)

ds_train_ = image_dataset_from_directory(
    train_dir,
    labels = 'inferred',
    label_mode = 'categorical',
    image_size = [128, 128],
    interpolation = 'nearest',
    batch_size = 64,
    shuffle = True,
)

ds_valid_ = image_dataset_from_directory(
    train_dir,
    labels = 'inferred',
    label_mode = 'categorical',
    image_size = [128, 128],
    interpolation = 'nearest',
    batch_size = 64,
    shuffle = False,
)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Training directory: /content/drive/MyDrive/Identify_planet/train
Validation directory: /content/drive/MyDrive/Identify_planet/valid
Found 368 files belonging to 8 classes.
Found 368 files belonging to 8 classes.


In [7]:
def convert_to_float(image, label):
    image = tf.image.convert_image_dtype(image, dtype = tf.float32)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = (
    ds_train_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size = AUTOTUNE)
)
ds_valid = (
    ds_valid_
    .map(convert_to_float)
    .cache()
    .prefetch(buffer_size = AUTOTUNE)
)

In [9]:
pretrained_base = VGG16(weights = 'imagenet',include_top = False, input_shape = (128,128,3))

pretrained_base.trainable = False

58889256/58889256 [==============================] - 4s 0us/step


In [11]:
#build
model = keras.Sequential([
    pretrained_base,
    layers.Flatten(),
    layers.Dense(256, activation = 'relu'),
    layers.Dropout(0.5),
    layers.Dense(8, activation = 'softmax')
])
#compile
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

#train
history = model.fit(
    ds_train,validation_data = ds_valid,
    epochs = 30,
    verbose = 1
)


Epoch 1/30
6/6 [==============================] - 8s 1s/step - loss: 1.5180 - accuracy: 0.4837 - val_loss: 0.3347 - val_accuracy: 0.8614
Epoch 2/30
6/6 [==============================] - 1s 165ms/step - loss: 0.3888 - accuracy: 0.8370 - val_loss: 0.1182 - val_accuracy: 1.0000
Epoch 3/30
6/6 [==============================] - 1s 155ms/step - loss: 0.2000 - accuracy: 0.9321 - val_loss: 0.0578 - val_accuracy: 1.0000
Epoch 4/30
6/6 [==============================] - 1s 157ms/step - loss: 0.1165 - accuracy: 0.9538 - val_loss: 0.0261 - val_accuracy: 1.0000
Epoch 5/30
6/6 [==============================] - 1s 155ms/step - loss: 0.0551 - accuracy: 0.9864 - val_loss: 0.0135 - val_accuracy: 1.0000
Epoch 6/30
6/6 [==============================] - 1s 156ms/step - loss: 0.0358 - accuracy: 0.9918 - val_loss: 0.0089 - val_accuracy: 1.0000
Epoch 7/30
6/6 [==============================] - 1s 156ms/step - loss: 0.0255 - accuracy: 1.0000 - val_loss: 0.0061 - val_accuracy: 1.0000
Epoch 8/30
6/6 [=======

In [12]:
labels = ['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune']

In [13]:
#preprocess image
def preprocess_img(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels = 3)
    image = tf.image.resize(image, [128,128])
    image = tf.cast(image, tf.float32) / 255.0
    return image

In [14]:
def predict_class(image_path):
    image = preprocess_img(image_path)
    image = tf.expand_dims(image, axis = 0)
    predictions = model.predict(image)

    predicted_index = tf.argmax(predictions[0]).numpy()
    predicted_class = labels[predicted_index]
    return predicted_class

In [19]:
#test
image_path = '/content/drive/MyDrive/Identify_planet/test/Saturn/Saturn729_jpg.rf.48052b68a5fc03dc5c62239e5081b7c2.jpg'

planet = predict_class(image_path)
print("The predicted planet is: ", planet)



1/1 [==============================] - 0s 20ms/step
The predicted planet is:  Saturn
